<hr style="border-width:4px; border-style:solid; border-color:coral"></hr>

# Introduction to MPI

<hr style="border-width:4px; border-style:solid; border-color:coral"></hr>

* [Demo 0](#demo0) (Almost) the simplest MPI example.

* [Demo 1](#demo1) Pass a string message between processors. 

* [Demo 2](#demo2) Pass a scalar data between processors 

* [Demo 3](#demo3) Pass an array between processors 

In [1]:
%matplotlib notebook
%pylab

Using matplotlib backend: nbAgg
Populating the interactive namespace from numpy and matplotlib


<a id="demo0"></a>

<hr style="border-width:4px; border-style:solid"></hr>

## Demo 0 - (Almost) the simplest MPI code
<hr style="border-width:4px; border-style:solid"></hr>

This example illustrates how to initialize and finalize the MPI library.  In this example, we also print out the rank of each processor. 

The basic requirement to use MPI are to include the `mpi.h` header file.  Then to compile, we have to use the `mpicc` rather than `gcc`.  

In [2]:
%%file demo_00.c

#include <mpi.h>
#include <stdio.h>

void main(int argc, char** argv)
{
    MPI_Init(&argc, &argv);
    
    int my_rank;
    MPI_Comm_rank(MPI_COMM_WORLD, &my_rank);

    printf("Rank %d\n",my_rank);

    MPI_Finalize();
}

Writing demo_00.c


In [3]:
%%bash

rm -rf demo_00

mpicc -o demo_00 demo_00.c

mpirun -n 4 demo_00

Rank 0
Rank 1
Rank 3
Rank 2


<a id="demo1"></a>

<hr style="border-width:4px; border-style:solid"></hr>

## Demo 1 - Pass a message between processors

<hr style="border-width:4px; border-style:solid"></hr>

In this example, we pass a simple string message from processor 0  to other ranks. 

In [14]:
%%file demo_01.c

#include <stdio.h>
#include <string.h>
#include <mpi.h>

void main(int argc, char** argv)
{
    MPI_Init(&argc, &argv);

    int my_rank, nprocs;
    MPI_Comm_rank(MPI_COMM_WORLD, &my_rank);
    MPI_Comm_size(MPI_COMM_WORLD, &nprocs);

    int tag = 0;
    if (my_rank == 0)
    {
        for(int p = 1; p < nprocs; p++)
        {
            int dest = p;
            char message[100];
            sprintf(message,"Hello processor %d from processor 0!",p);
            MPI_Send(message,strlen(message)+1,MPI_CHAR,dest,tag,MPI_COMM_WORLD);
        }
    }
    else
    {
        int sender = 0;
        int count;
        char message[100];
        MPI_Recv(message,100,MPI_CHAR,sender,tag,MPI_COMM_WORLD,MPI_STATUS_IGNORE);
        printf("Rank %d received message : \"%s\"\n",my_rank,message);
    }
    
    MPI_Finalize();
}

Overwriting demo_01.c


In [13]:
%%bash

rm -rf demo_01

mpicc -o demo_01 demo_01.c

mpirun -n 8 demo_01

Rank 1 received message : "Hello processor 1 from processor 0!"
Rank 3 received message : "Hello processor 3 from processor 0!"
Rank 4 received message : "Hello processor 4 from processor 0!"
Rank 5 received message : "Hello processor 5 from processor 0!"
Rank 6 received message : "Hello processor 6 from processor 0!"
Rank 7 received message : "Hello processor 7 from processor 0!"
Rank 2 received message : "Hello processor 2 from processor 0!"


<a id="demo2"></a>

<hr style="border-width:4px; border-style:solid"></hr>

## Demo 2 - Pass scalar data between processors

<hr style="border-width:4px; border-style:solid"></hr>

In this example, we pass a scalar value `data` from processor 0 to each of the other ranks.  Rank 0 sends each rank an integer equal to its rank.  

In [7]:
%%file demo_02.c

#include <mpi.h>
#include <stdio.h>    /* For IO */

void main(int argc, char** argv)
{
    MPI_Init(&argc, &argv);

    int rank, nprocs;
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    MPI_Comm_size(MPI_COMM_WORLD, &nprocs);

    /* Hardwire length of array */
    int tag = 0;
    int data;
    if (rank == 0)
    {
        /* Allocate and initialize a big array for all data */
        for(int p = 1; p < nprocs; p++)
        {
            int dest = p;
            int data = p;
            MPI_Send(&data,1,MPI_DOUBLE,dest,tag,MPI_COMM_WORLD);
        }
    }
    else
    {
        int source = 0;        
        MPI_Recv(&data,1,MPI_DOUBLE,source,tag,MPI_COMM_WORLD,MPI_STATUS_IGNORE);
    }
    printf("Rank %d : (%d)\n",rank,data);
        
    MPI_Finalize();
}


Writing demo_02.c


In [8]:
%%bash

rm -rf demo_02

mpicc -o demo_02 demo_02.c
mpirun -n 4 demo_02

Rank 0 : (0)
Rank 1 : (1)
Rank 2 : (2)
Rank 3 : (3)


<a id="demo3"></a>

<hr style="border-width:4px; border-style:solid"></hr>

## Demo 3 - Pass an array between processors 

<hr style="border-width:4px; border-style:solid"></hr>

Rank 0 allocates an array of length 64, and initializes the array with integers $0,1,2,...64$.  Then rank 0 passes blocks of length $N/nprocs$ to the other ranks.  The program then reports the starting and ending values for the local arrays received by that rank.

In [15]:
%%file demo_03.c

#include <mpi.h>
#include <stdio.h>    /* For IO */
#include <stdlib.h>   /* For malloc */

void main(int argc, char** argv)
{
    MPI_Init(&argc, &argv);

    int rank, nprocs;
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    MPI_Comm_size(MPI_COMM_WORLD, &nprocs);

    /* Hardwire length of array */
    int N = 64;
    int Nlocal = N/nprocs;

    double *x;
    int tag = 0;
    if (rank == 0)
    {
        /* Allocate and initialize a big array for all data */
        x = malloc((N+1)*sizeof(double));
        for(int i = 0; i < N+1; i++)
        {
            x[i] = i;
        }
        for(int p = 1; p < nprocs; p++)
        {
            int dest = p;
            MPI_Send(&x[p*Nlocal],Nlocal+1,MPI_DOUBLE,dest,tag,MPI_COMM_WORLD);
        }
    }
    else
    {
        int source = 0;
        x = malloc((Nlocal+1)*sizeof(double));
        MPI_Recv(x,Nlocal+1,MPI_DOUBLE,source,tag,MPI_COMM_WORLD,MPI_STATUS_IGNORE);
    }
    printf("Rank %d : (%2.0f,...,%2.0f)\n",rank,x[0],x[Nlocal]);
        
    free(x);
    
    MPI_Finalize();
}

Overwriting demo_03.c


In [16]:
%%bash

rm -rf demo_03

mpicc -o demo_03 demo_03.c
mpirun -n 4 demo_03

Rank 0 : ( 0,...,16)
Rank 1 : (16,...,32)
Rank 2 : (32,...,48)
Rank 3 : (48,...,64)


<a id="demo4"></a>

<hr style="border-width:4px; border-style:solid"></hr>

## Demo 4

<hr style="border-width:4px; border-style:solid"></hr>